In [1]:
import random as rdm
import pandas as pd
import numpy as np

In [96]:
df_disease = pd.read_csv("doencas.csv")
df_disease.head()

,id,genero,data,cidade,estado,doenca
0,1,Female,31/01/2000,Brooklyn,New York,Virose
1,2,Female,16/10/1999,Topeka,Kansas,Zika
2,3,Female,10/10/1999,Schenectady,New York,Virose
3,4,Female,09/01/2000,New York City,New York,Conjuntivite
4,5,Male,03/11/1999,Syracuse,New York,Conjuntivite


In [119]:
df_public = pd.read_csv("background.csv")
df_public.reset_index(inplace=True)
df_public.rename(columns={'index': 'id'}, inplace=True)
df_public.head()

,id,nome,genero,data,cidade,estado
0,0,Alessandro Cowterd,Male,27/12/1999,New York City,New York
1,1,Cathie Bernhardi,Female,07/10/1999,Brooklyn,New York
2,2,Dorothee McCluskey,Female,14/12/1999,Buffalo,New York
3,3,Mace Scrine,Male,13/03/2000,New York City,New York
4,4,Ethyl Suttie,Female,18/11/1999,New York City,New York


# K-Anonimato

## Joining the locality attributes

In [102]:
ids_disease = df_disease.id
cities_disease = df_disease.cidade
states_disease = df_disease.estado

In [121]:
ids_public = df_public.id
cities_public = df_public.cidade
states_public = df_public.estado

In [125]:
locality_disease = []
for i in range(len(cities_disease)):
    locality_disease.append([ids_disease[i], str('{}, {}'.format(cities_disease[i], states_disease[i]))])

In [126]:
locality_public = []
for i in range(len(cities_public)):
    locality_public.append([ids_public[i], str('{}, {}'.format(cities_public[i], states_public[i]))])

In [127]:
df_locality_disease = pd.DataFrame(locality_disease, columns=['id', 'localidade'])
df_locality_disease.head()

,id,localidade
0,1,"Brooklyn, New York"
1,2,"Topeka, Kansas"
2,3,"Schenectady, New York"
3,4,"New York City, New York"
4,5,"Syracuse, New York"


In [128]:
df_locality_public = pd.DataFrame(locality_public, columns=['id', 'localidade'])
df_locality_public.head()

,id,localidade
0,0,"New York City, New York"
1,1,"Brooklyn, New York"
2,2,"Buffalo, New York"
3,3,"New York City, New York"
4,4,"New York City, New York"


## Joining the dataframes

In [137]:
join_dataset_disease = pd.concat([df_disease, df_locality_disease], axis=1, join='inner')
join_dataset_disease.head()

,id,genero,data,cidade,estado,doenca,id,localidade
0,1,Female,31/01/2000,Brooklyn,New York,Virose,1,"Brooklyn, New York"
1,2,Female,16/10/1999,Topeka,Kansas,Zika,2,"Topeka, Kansas"
2,3,Female,10/10/1999,Schenectady,New York,Virose,3,"Schenectady, New York"
3,4,Female,09/01/2000,New York City,New York,Conjuntivite,4,"New York City, New York"
4,5,Male,03/11/1999,Syracuse,New York,Conjuntivite,5,"Syracuse, New York"


In [136]:
join_dataset_public = pd.concat([df_public, df_locality_public], axis=1, join='inner')
join_dataset_public.head()

,id,nome,genero,data,cidade,estado,id,localidade
0,0,Alessandro Cowterd,Male,27/12/1999,New York City,New York,0,"New York City, New York"
1,1,Cathie Bernhardi,Female,07/10/1999,Brooklyn,New York,1,"Brooklyn, New York"
2,2,Dorothee McCluskey,Female,14/12/1999,Buffalo,New York,2,"Buffalo, New York"
3,3,Mace Scrine,Male,13/03/2000,New York City,New York,3,"New York City, New York"
4,4,Ethyl Suttie,Female,18/11/1999,New York City,New York,4,"New York City, New York"


## Removing attributes

In [135]:
dataset_disease = join_dataset_disease.drop(columns = ['id', 'estado', 'cidade'])
dataset_disease.describe()

,genero,data,doenca,localidade
count,400,400,400,400
unique,2,160,5,24
top,Male,19/03/2000,Dengue,"New York City, New York"
freq,204,6,87,103


In [133]:
dataset_public = join_dataset_public.drop(columns = ['id', 'estado', 'cidade'])
dataset_public.describe()

,nome,genero,data,localidade
count,2000,2000,2000,2000
unique,2000,2,182,24
top,Kerr Yannikov,Male,13/01/2000,"New York City, New York"
freq,1,1032,22,519


## Date generalization function

In [19]:
def date_generalization(dataset, level):
    if level == 1:
        # date '**/MM/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**' + dataset.iloc[data]['data'][2:10])
    elif level == 2:
        # date '**/**/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**' + dataset.iloc[data]['data'][5:10])
    elif level == 3:
        # date'**/**/****'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**/****')
        
    return dataset

In [170]:
def locality_generalization(dataset, level):
    if level == 0:
        # locality 'city'
        for data in range(len(dataset)):
            place = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = place[0] + "," + place[1]
#             print(place[0], place[1])
    elif level == 1:
        # locality 'state,'
        for data in range(len(dataset)):
            place = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**," + place[1]
    elif level == 2:
        # locality '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = '*'
        
    return dataset

In [155]:
def gender_generalization(dataset, level):
    if level == 1:
        # gender '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['genero'] = '*'
            
    return dataset

## Precision metric

$precision (D, Na, h, HGVA) = 1 - \frac{\sum_{i=1}^{Na} \sum_{j=1}^{|D|} \frac{h}{HGVA_i}}{|D| * Na}
$

- D: dataset
- Na: number of semi-identifier attributes
- h: height of attribute value generalization hierarchy Ai after generalization
- |HGVAi|: maximum height of hierarchy

In [156]:
def precision(D, Na, HGVA):
    h = []
    summation = 0
    for j in range(len(D)):
        data = D.iloc[j]["data"].count('*')
        localidade = D.iloc[j]["localidade"].count('*')
        genero = D.iloc[j]["genero"].count('*')
        
        h_data = 0
        if data == 2:
            h_data = 1
        elif data == 4:
            h_data = 2
        elif data == 8:
            h_data = 3
            
        h_localidade = 0
        if   localidade == 1:
            h_localidade = 2
        elif localidade == 2:
            h_localidade = 1
            
        h_genero = 0
        if genero == 1:
            h_genero = 1
            
        h.append([h_data, h_genero, h_localidade])
    
    for i in range(Na):
        for j in range(len(D)):
            summation += (h[j][i] / (HGVA[i]))
    
    return 1 - (summation / (len(D) * Na))

# Tree generalization

In [157]:
def treeGeneralization():
    # data, genero, localidade
    return [(a,b,c) for a in range(4) for b in range(2) for c in range(3)]

# Verification Delta Presence

In [201]:
def verifyDeltaPresence(dataset, public, minimum, maximum, level_anonymization):
    new_dataset = dataset.copy()
    new_dataset.drop(columns=["doenca"])
    
    new_public_dataset = public.copy()
    new_public_dataset.drop(columns=["nome"])
    
    
    current_tree_generalization = treeGeneralization()[level_anonymization]
    new_public_dataset = date_generalization(new_public_dataset.copy(), current_tree_generalization[0])
    new_public_dataset = gender_generalization(new_public_dataset, current_tree_generalization[1])
    new_public_dataset = locality_generalization(new_public_dataset, current_tree_generalization[2])
    
    grouped_public_dataset = new_public_dataset.groupby(['data','genero','localidade'])
    grouped_dataset = new_dataset.groupby(['data','genero','localidade'])
    
    for i in range(len(new_public_dataset)):
        registry = (new_public_dataset.iloc[i]['data'], 
                    new_public_dataset.iloc[i]['genero'], 
                    new_public_dataset.iloc[i]['localidade'])

        try:
            anonymized_amount = len(grouped_dataset.groups[registry])
            public_amount = len(grouped_public_dataset.groups[registry])
            delta = anonymized_amount / public_amount
    #         print(delta)

            if delta > maximum or delta < minimum:
                return (False, 0) # passe para o proximo nivel de generalização
        except KeyError:
            pass
        
    return (True, delta)

# Generating anonymization

In [204]:
def deltaPresence(dataset, public, minimum, maximum):
    
    current_level_generalization = 0
    
    for current_tree_generalization in treeGeneralization():
        
        current_dataset_generalization = date_generalization(dataset.copy(), 
                                                             current_tree_generalization[0])
        
        current_dataset_generalization = gender_generalization(current_dataset_generalization, 
                                                               current_tree_generalization[1])
        
        current_dataset_generalization = locality_generalization(current_dataset_generalization, 
                                                                 current_tree_generalization[2])
             
        return_verification = verifyDeltaPresence(current_dataset_generalization, 
                                                  public, 
                                                  minimum, 
                                                  maximum, 
                                                  current_level_generalization)
        
        if (return_verification[0]):
            print(return_verification[1])
            ###
            # Salvar dataframe
#             calculate_precision = float("{0:.4f}"
#                                         .format(precision(current_dataset_generalization, 3, [3, 1, 2])))
            
#             print("{} => {}"
#                   .format(current_tree_generalization, calculate_precision))
            return current_dataset_generalization

        current_level_generalization += 1
            
    return current_dataset_generalization

In [205]:
generalized_dataset = deltaPresence(dataset_disease.copy(), dataset_public.copy(), 0.10, 0.50)

0.19607843137254902


In [206]:
generalized_dataset

,genero,data,doenca,localidade
0,Female,**/01/2000,Virose,*
1,Female,**/10/1999,Zika,*
2,Female,**/10/1999,Virose,*
3,Female,**/01/2000,Conjuntivite,*
4,Male,**/11/1999,Conjuntivite,*
5,Female,**/01/2000,Gripe,*
6,Male,**/11/1999,Zika,*
7,Female,**/11/1999,Gripe,*
8,Male,**/01/2000,Gripe,*
9,Male,**/01/2000,Conjuntivite,*
